# COVID-19 MIDS Collaboration 

## Data Import: COVID-19 Data

This Jupyter Notebook reads in raw data as csv files and exports them as [pickle files for faster loading](https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e). 

### Data sources

COVID-19 data: 

### Set up environment 

In [1]:
# Import packages
import pandas as pd # to read in csv
import numpy as np # to process data
import pickle # to pickle files


Here, we import our dataset into a pandas dataframe and add a "o_" prefix to all columsn to demarcate which columns were the originals. 

In [2]:
#Read in data & add "o_" prefix to demarcate which columns were the originals 
# covid cases data
covid_df = pd.read_csv("../Data_raw/us-counties.csv").add_prefix('o_') 


### Inspect data

#### Count records

Our dataset tracking COVID-19 cases and deaths by county has 61,971 rows and 6 columns. 

In [3]:
# count rows and columns
covid_df.shape

(61971, 6)

#### Check and convert datatypes

We have:

* one string column tracking the date (which needs to be converted to a date object)
* 3 float columns tracking location 
    * county and state are correctly stored as strings
    * fips is a float and should be converted to a category - it should also be split into county and state codes 
* 2 integer columns tracking COVID-19 cases and deaths

In [4]:
# check data types 
covid_df.dtypes

o_date       object
o_county     object
o_state      object
o_fips      float64
o_cases       int64
o_deaths      int64
dtype: object

In [5]:
covid_df.head()

,o_date,o_county,o_state,o_fips,o_cases,o_deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [6]:
# create new version of column as a datetime object
covid_df["date"] = pd.to_datetime(covid_df["o_date"])
# check conversion 
covid_df.dtypes

o_date              object
o_county            object
o_state             object
o_fips             float64
o_cases              int64
o_deaths             int64
date        datetime64[ns]
dtype: object

In [7]:
# create new version of fips column as nullable integer
covid_df["fips"] = covid_df["o_fips"].astype('category')
# create new version of fips column as string 
covid_df["fips_str"] = covid_df["o_fips"].astype('Int64').astype('str')
# check conversion 
covid_df.dtypes

o_date              object
o_county            object
o_state             object
o_fips             float64
o_cases              int64
o_deaths             int64
date        datetime64[ns]
fips              category
fips_str            object
dtype: object

In [8]:
# extract state FIPS and convert to category 
covid_df["state_fips"] = covid_df["fips_str"].apply(lambda x: x[:-3]).astype('category')
covid_df["state_fips"].head()

0    53
1    53
2    53
3    17
4    53
Name: state_fips, dtype: category
Categories (52, object): [, 1, 10, 11, ..., 56, 6, 8, 9]

In [9]:
# extract county FIPS and convert to category 
covid_df["county_fips"] = covid_df["fips_str"].apply(lambda x: x[-3:]).astype('category')
covid_df["county_fips"].head()

0    061
1    061
2    061
3    031
4    061
Name: county_fips, dtype: category
Categories (284, object): [001, 003, 005, 006, ..., 820, 830, 840, nan]

In [10]:
# visual check of column conversions - these worked 
covid_df.head()

,o_date,o_county,o_state,o_fips,o_cases,o_deaths,date,fips,fips_str,state_fips,county_fips
0,2020-01-21,Snohomish,Washington,53061.0,1,0,2020-01-21,53061.0,53061,53,061
1,2020-01-22,Snohomish,Washington,53061.0,1,0,2020-01-22,53061.0,53061,53,061
2,2020-01-23,Snohomish,Washington,53061.0,1,0,2020-01-23,53061.0,53061,53,061
3,2020-01-24,Cook,Illinois,17031.0,1,0,2020-01-24,17031.0,17031,17,031
4,2020-01-24,Snohomish,Washington,53061.0,1,0,2020-01-24,53061.0,53061,53,061


### Check missingness

We have 807 missing FIPS codes, but no other missing data (at least that is coded as NaNs).

In [11]:
covid_df.isnull().sum(axis = 0)

o_date           0
o_county         0
o_state          0
o_fips         807
o_cases          0
o_deaths         0
date             0
fips           807
fips_str         0
state_fips       0
county_fips      0
dtype: int64

3 counties have missing FIPS codes.

In [12]:
covid_df["o_county"][covid_df["o_fips"].isnull()].unique()

array(['New York City', 'Unknown', 'Kansas City'], dtype=object)

New York City is comprised of 5 counties (1 for each of the 5 boroughs); these each have their own FIPS codes: New York County (Manhattan), Kings County (Brooklyn), Bronx County (The Bronx), Richmond County (Staten Island), and Queens County (Queens). 

In [13]:
# show unique state for counties coded as "Unknown"
covid_df["o_state"][covid_df["o_county"] == "New York City"].unique()

array(['New York'], dtype=object)

In [14]:
# show unique fips codes for counties coded as "New York City"
covid_df["o_fips"][covid_df["o_county"] == "New York City"].unique()

array([nan])

Kansas City, Missouri also straddles multiple counties which would explain why it does not have a FIPS code. 

In [15]:
# show unique states for counties coded as "Kansas City"
covid_df["o_state"][covid_df["o_county"] == "Kansas City"].unique()

array(['Missouri'], dtype=object)

In [16]:
# show unique fips codes for counties coded as "Kansas City"
covid_df["o_fips"][covid_df["o_county"] == "Kansas City"].unique()

array([nan])

Other rows are missing FIPS codes because they lack a county (it is marked as "Unknown"). Some would make sense - for example, Puerto Rico only has a state FIPS code. 

In [17]:
# show unique states for counties coded as "Unknown"
covid_df["o_state"][covid_df["o_county"] == "Unknown"].unique()

array(['Rhode Island', 'New Jersey', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'Maine', 'Massachusetts', 'Louisiana', 'Kentucky',
       'Nevada', 'Tennessee', 'Arkansas', 'Georgia', 'Missouri',
       'Minnesota', 'California', 'Colorado', 'Florida', 'Hawaii',
       'Illinois', 'Vermont', 'Idaho', 'Michigan', 'Ohio', 'Utah',
       'Mississippi', 'Northern Mariana Islands', 'Nebraska',
       'Connecticut', 'South Dakota', 'Indiana', 'Alaska', 'Arizona',
       'New Mexico', 'New York', 'Pennsylvania', 'Virginia',
       'New Hampshire', 'Washington', 'Delaware', 'Kansas',
       'North Dakota', 'Maryland'], dtype=object)

In [18]:
# show unique fips codes for counties coded as "Unknown"
covid_df["o_fips"][covid_df["o_county"] == "Unknown"].unique()

array([nan])

The missingness in FIPS codes is a major issue since this is the geographic identifier we would use to join these data to other datsets. We will likely need to accurately rectify this missingness in order to accurately combine our data. These don't seem to be erroneous data for the most part. Rather, the missing points reflect when a data point is at the city (rather than county) level and/or if there is no county for that region. 

In [19]:
# replace rows with missing county with NaN 
covid_df["county"] = covid_df["o_county"].replace("Unknown", None)

### Generate summary statistics

The mins and maxes look reasonable for COVID-19 mins and maxes. The distributions are remarkable but also reasonable - we clearly have some outliers on the high end (New York is likely one of those). 

We have no missing data in the text columns. We have data for 86 unique days, in 55 "states" (so the data also include territories), and 1,627 counties. In the original county column, we see there are 1,628 unique values since the NaNs used to be coded as "Unknown".

We can see that we have some missing data in our FIPS codes column and that we have 2,708 unique codes. 

Our earliest date was January 21st, 2020. Our last was April 15th, 2020. 

In [33]:
# display summary statistics
covid_df.describe(include = 'all')

,o_date,o_county,o_state,o_fips,o_cases,o_deaths,date,fips,fips_str,state_fips,county_fips,county
count,61971,61971,61971,61164.000000,61971.000000,61971.000000,61971,61164.0,61971,61971,61971,61971
unique,86,1628,55,NaN,NaN,NaN,86,2708.0,2709,52,284,1627
top,2020-04-15,Washington,Texas,NaN,NaN,NaN,2020-04-15 00:00:00,53061.0,nan,48,001,Washington
freq,2722,772,3685,NaN,NaN,NaN,2722,86.0,807,3685,1119,777
first,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-21 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-15 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,29601.488980,121.685353,4.025415,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,15528.488936,1520.051958,82.937968,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,1001.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,17179.000000,2.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


Now that we have verified our column conversion worked, we drop the columns for which we had added additional columns with the correct data type and remove the "o_" prefix since we no longer need to compare original versus converted columns. Columns are reordered to match the original ordering in the dataset.

In [23]:
# drop, rename, and reorder columns
converted_covid_df = covid_df.drop(columns = ["o_date", "o_fips", "o_county", "fips_str"]).rename(columns = lambda x: x.replace('o_', ''))[["date", "fips", "state_fips", "county_fips","county", "state", "cases", "deaths"]]
# visual inspection
converted_covid_df.head()

,date,fips,state_fips,county_fips,county,state,cases,deaths
0,2020-01-21,53061.0,53,061,Snohomish,Washington,1,0
1,2020-01-22,53061.0,53,061,Snohomish,Washington,1,0
2,2020-01-23,53061.0,53,061,Snohomish,Washington,1,0
3,2020-01-24,17031.0,17,031,Cook,Illinois,1,0
4,2020-01-24,53061.0,53,061,Snohomish,Washington,1,0


### Check duplicates

There are no perfectly duplicated rows. 

In [24]:
sum(converted_covid_df.duplicated())

0

There are 2,721 unique county-state pairs. 

In [25]:
# create column to store county-state pairs
converted_covid_df["county_state"] = (converted_covid_df["county"] + ", " + converted_covid_df["state"])
# count unique county-state pairs
len(converted_covid_df["county_state"].unique())

2721

Suffolk, Massachusetts appears the most frequently, which is unexpected. It has 20 more entries than there are days in the dataset. The next county is Snohomish, Washington occurs 86 times, which could represent one record per day. Some areas appear only once. 

In [26]:
converted_covid_df["county_state"].value_counts()

Suffolk, Massachusetts     106
Snohomish, Washington       86
Cook, Illinois              83
Orange, California          82
Los Angeles, California     81
                          ... 
Butte, South Dakota          1
Smith, Kansas                1
Harrison, Ohio               1
Washita, Oklahoma            1
Boundary, Idaho              1
Name: county_state, Length: 2721, dtype: int64

Suffolk, Massachusettes has only 75 reported days. 

In [27]:
len(converted_covid_df["date"][converted_covid_df["county_state"] == "Suffolk, Massachusetts"].unique())

75

We can see that the maximum number of unique dates is for Snohomish, Washington (86). We will need to flatten our file so that it is at the date level - so that there is one date per county/state pair. 

In [28]:
# show first 5 rows of dataframe
converted_covid_df.groupby('county_state').date.nunique().reset_index().sort_values('date', ascending = 0).head(5)

,county_state,date
2255,"Snohomish, Washington",86
548,"Cook, Illinois",83
1845,"Orange, California",82
1462,"Los Angeles, California",81
1528,"Maricopa, Arizona",81


In [29]:
# show first 5 rows of dataframe
converted_covid_df.groupby('county_state').date.nunique().reset_index().sort_values('date', ascending = 0).tail(5)

,county_state,date
1799,"Nome Census Area, Alaska",1
27,"Alcona, Michigan",1
1731,"Morrill, Nebraska",1
1802,"Norman, Minnesota",1
210,"Boundary, Idaho",1


### Convert to appropriate level

Here, we flatten the file so that there is one row per county-state pair for each day. We sum over the cases and deaths for that day to generate a new cases and deaths column. 

In [30]:
# create new dataframe flattened to county-state pair and date level 
flattened_covid_df = converted_covid_df.groupby(['county_state', 'date']).agg(
    {
     'state': 'first',
     'county': 'first',
     'fips': np.unique,
     'county_fips': np.unique,
     'state_fips':np.unique,
     'cases':np.sum, 
     'deaths':np.sum
    }).reset_index().explode('fips').explode('county_fips')

In [31]:
# check new dataframe 
flattened_covid_df.head()

,county_state,date,state,county,fips,county_fips,state_fips,cases,deaths
0,"Abbeville, South Carolina",2020-03-19,South Carolina,Abbeville,45001,001,[45],1,0
1,"Abbeville, South Carolina",2020-03-20,South Carolina,Abbeville,45001,001,[45],1,0
2,"Abbeville, South Carolina",2020-03-21,South Carolina,Abbeville,45001,001,[45],1,0
3,"Abbeville, South Carolina",2020-03-22,South Carolina,Abbeville,45001,001,[45],1,0
4,"Abbeville, South Carolina",2020-03-23,South Carolina,Abbeville,45001,001,[45],1,0


In [34]:
flattened_covid_df.describe(include = 'all')

,county_state,date,state,county,fips,county_fips,state_fips,cases,deaths
count,65523,65523,65523,65523,62940.0,65523,65523,65523.000000,65523.000000
unique,2721,86,55,1627,2708.0,284,644,NaN,NaN
top,"Suffolk, Massachusetts",2020-04-15 00:00:00,Georgia,Union,25025.0,nan,[48],NaN,NaN
freq,292,2866,3818,1409,168.0,2583,3685,NaN,NaN
first,NaN,2020-01-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2020-04-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.773652,4.658761
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1521.337131,81.936473
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000


### Pickle data

In [32]:
pickle.dump(flattened_covid_df, open( "../Data_pkl/flattened_covid_df.pkl", "wb" ) )